<a href="https://colab.research.google.com/github/mrzeszewski/Podstawy_Big_Data/blob/main/notatnik/Wprowadzenie_do_GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Inicjalizacja GEE

In [2]:
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-mrzeszewski')

## Kolekcje danych

W GEE dane zabrane są w kolekcje obrazów (ImageCollection) i obiektów (FeatureCollection) - raster/wektor

In [17]:
# ImageCollection - warstwy rastrowe
populacja = ee.ImageCollection("WorldPop/GP/100m/pop") #kolekcja obrazów

mapka=geemap.Map() #incjalizacja mapy - wywołanie resetuje obiekt

#parametry wizualne w postaci słownika
vis_params={
    'min':0, #wartość rastra przypisana pierwszemu kolorowi z palety
    'max':5, #wartość rastra przypisana ostatniemu kolorowi z palety
    #'palette':['red','green','blue'] #piewszy sposób określenia palety - nazwy kolorów
    # 'palette':'inferno' #nazwa palety
    'palette':'viridis' #inna paleta
    #można też kolory szesnatkowo (RRGGBB lub RRGGBBAA) wg. CSS3
}

mapka.addLayer(populacja,vis_params=vis_params,name="Populacja") #dodanie warstwy do mapy

mapka #wyświetlenie mapy



Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [20]:
#FeatureCollection - warstwy wektorowe

granice_panstw=ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0")

mapka=geemap.Map() #incjalizacja mapy - wywołanie resetuje obiekt

vis_params={
    'color':'blue',
    'width':2,
    'lineType':'solid',
    'fillColor':'00000000'
}

mapka.addLayer(granice_panstw,vis_params=vis_params,name="Granice państw") #dodanie warstwy do mapy

mapka #wyświetlenie mapy

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Pozyskiwanie informacji o zbiorze danych

In [28]:
print(populacja.size().getInfo()) #funkcja getinifo() powoduje wykonanie działania na serwerze

display(populacja.size()) #display to funkcja wyższego rzędu niz print - "próbuje" wyświetlić dane w jak najlepszy sposób

display(populacja.limit(2).getInfo())

5221


{'type': 'ImageCollection',
 'bands': [],
 'version': 1641990785000383,
 'id': 'WorldPop/GP/100m/pop',
 'properties': {'system:visualization_0_min': '0.0',
  'type_name': 'ImageCollection',
  'keywords': ['demography', 'population', 'worldpop'],
  'thumb': 'https://mw1.google.com/ges/dd/images/WorldPop_GP_100m_pop_thumb.png',
  'description': '<p>Global high-resolution, contemporary data on human population distributions are\na prerequisite for the accurate measurement of the impacts of population\ngrowth, for monitoring changes, and for planning interventions. The WorldPop\nproject aims to meet these needs through the provision of detailed and open\naccess population distribution datasets built using transparent and\npeer-reviewed approaches.</p><p>Full details on the methods and datasets used in constructing the\ndata, along with open access publications, are provided on the\n<a href="https://www.worldpop.org.uk">WorldPop website</a>. In brief, recent census-based\npopulation counts 

In [33]:
#Pozyskiwanie informacji o zbiorze obiektów

# display(granice_panstw.limit(2).getInfo())

display(granice_panstw.propertyNames()) #nazwy atrybutów dla kolekcji

display(granice_panstw.first().propertyNames()) #nazwy atrybutów dla pierwszego obiektu z kolekcji

display(granice_panstw.aggregate_array('STATUS')) #wartości dla kolumny STATUS dla wszystkich obiektów

display(granice_panstw.aggregate_array('ADM0_NAME'))

## FILTROWANIE DANYCH

In [42]:
copernicus = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')

# display(copernicus.size())

roi = ee.Geometry.Rectangle([20.85, 52.08, 21.27, 52.37]) #przybliżony BBOX  Warszawy

warszawa=copernicus.filterBounds(roi) #filtruje kolekcje do geometrii (zdefiniowane jako roi)

warszawa2019=warszawa.filterDate('2019-01-01','2020-01-01') #filtruje kolekcje do okresu daty - rok 2019

warszawa2024=warszawa.filterDate('2024-01-01','2025-01-01') #filtruje kolekcje do okresu daty - rok 2019


vis_params={
    'bands':['B4','B3','B2'], #wybór pasm B4 B3 i B2 to RGB - obraz widzialny
    'min':0,
    'max':3000
}

mapka=geemap.Map() #incjalizacja mapy - wywołanie resetuje obiekt

mapka.addLayer(warszawa2019,vis_params=vis_params, name="2019") #dodanie warstwy do mapy
mapka.addLayer(warszawa2024,vis_params=vis_params, name="2024")

# mapka.addLayer(roi,{'color':'red'},'roi') #dodanie warstwy do mapy

mapka.centerObject(roi,10) #ustawienie środka mapy na roi i poziom powiększenia 10

mapka #wyświetlenie mapy




Map(center=[52.22502849947438, 21.060000000000716], controls=(WidgetControl(options=['position', 'transparent_…